### Model Development

Import liberaries

In [2]:
import os, sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler

from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier

current_dir = os.getcwd()
# Append the parent directory to sys.path
parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)

# ignore warrning message
import warnings
warnings.filterwarnings('ignore')

In [3]:
from scripts.model_development import ModelPipeline

In [4]:
# load preprocessed datasets
fraud_data = pd.read_csv('../data/proccessed_fraud_data.csv')
credit_data = pd.read_csv('../data/creditcard.csv')

In [5]:
# print the first 5 rows of the data
fraud_data.head()

,user_id,signup_time,purchase_time,purchase_value,device_id,age,class,ip_int,country,hour_of_day,...,time_diff,transaction_frequency,average_velocity,source_Direct,source_SEO,browser_FireFox,browser_IE,browser_Opera,browser_Safari,sex_M
0,2,2015-01-11 03:47:13,2015-02-21 10:03:37,0.310345,FGBQNDNBETFJJ,0.120690,0,880217484,United States,10,...,0.0,1,0.0,0,1,0,0,0,0,0
1,4,2015-06-02 16:40:57,2015-09-26 21:32:16,0.220690,MKFUIVOHLJBYN,0.344828,0,2785906106,Switzerland,21,...,0.0,1,0.0,1,0,0,0,0,1,0
2,8,2015-05-28 07:53:06,2015-08-13 11:53:07,0.262069,SCQGQALXBUQZJ,0.120690,0,356056736,United States,11,...,0.0,1,0.0,0,1,0,0,0,0,1
3,12,2015-01-10 06:25:12,2015-03-04 20:56:37,0.179310,MSNWCFEHKTIOY,0.017241,0,2985180352,Mexico,20,...,0.0,1,0.0,0,0,0,0,0,1,1
4,16,2015-02-03 13:48:23,2015-03-12 12:46:23,0.000000,FROZWSSWOHZBE,0.241379,0,578312545,United States,12,...,0.0,1,0.0,1,0,0,1,0,0,1


In [6]:
fraud_data.transaction_frequency.unique()

array([1])

In [7]:
# Drop unnecessary columns
fraud_data = fraud_data.drop(['user_id', 'device_id', 'transaction_frequency','time_diff','average_velocity', 'ip_int', 'signup_time', 'purchase_time'], axis=1)

#### Frequency Encoding
In this method, I replace each category with its frequency in the fraud dataset. This can help maintain some information about the category without expanding the feature space too much.

In [8]:
# Frequency Encoding Example
frequency = fraud_data['country'].value_counts()
fraud_data['country_encoded'] = fraud_data['country'].map(frequency)

# Drop the original 'country' column
fraud_data = fraud_data.drop('country', axis=1)

# scale 'country_encoded' column
scaler = MinMaxScaler()
fraud_data['country_encoded'] = scaler.fit_transform(fraud_data[['country_encoded']])[:, 0]


In [9]:
fraud_data.to_csv('../data/final_preprocessed_fraud_data.csv', index=False)

In [10]:
fraud_data

,purchase_value,age,class,hour_of_day,day_of_week,source_Direct,source_SEO,browser_FireFox,browser_IE,browser_Opera,browser_Safari,sex_M,country_encoded
0,0.310345,0.120690,0,10,5,0,1,0,0,0,0,0,1.000000
1,0.220690,0.344828,0,21,5,1,0,0,0,0,1,0,0.013506
2,0.262069,0.120690,0,11,3,0,1,0,0,0,0,1,1.000000
3,0.179310,0.017241,0,20,2,0,0,0,0,0,1,1,0.019294
4,0.000000,0.241379,0,12,3,1,0,0,1,0,0,1,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
129141,0.503448,0.706897,0,7,2,1,0,0,0,0,1,1,0.054438
129142,0.075862,0.327586,0,7,3,1,0,0,1,0,0,0,0.062793
129143,0.165517,0.293103,0,23,4,1,0,1,0,0,0,0,0.125844
129144,0.393103,0.517241,0,20,2,1,0,0,0,0,0,1,1.000000


### Model Development for Fraud_detaction

In [11]:
# Example usage for fraud dataset:
fraud_data_file_path = '../data/final_preprocessed_fraud_data.csv'
fraud_pipeline = ModelPipeline('fraud', fraud_data_file_path)

2025/02/11 16:15:00 INFO mlflow.tracking.fluent: Experiment with name 'Fraud_Detection_Experiment' does not exist. Creating a new experiment.


In [12]:
# load data for model development
fraud_pipeline.load_data()

2025-02-11 16:15:38,404 - INFO - Loading fraud data from ../data/final_preprocessed_fraud_data.csv...
2025-02-11 16:15:38,572 - INFO - Data loading complete.
